# 用TensorFlow实现卷积神经网络（LeNet-5模型）

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
print(mnist.train.images.shape,mnist.train.labels.shape)
print(mnist.test.images.shape,mnist.test.labels.shape)

(55000, 784) (55000, 10)
(10000, 784) (10000, 10)


In [5]:
# 一，函数声明部分  

def weight_variable(shape):  
    # 初始化权重；截断正态分布，标准差为0.1，默认最大为1，最小为-1，均值为0 
        initial = tf.truncated_normal(shape, stddev=0.1)  
        return tf.Variable(initial)  
def bias_variable(shape):  
    # 初始化偏置值为0.1  
        initial = tf.constant(0.1, shape=shape)  
        return tf.Variable(initial)  
def conv2d(x, W):    
    # 卷积操作；x为做卷积的输入图像，W为相当于CNN中的卷积核，，strides是滑动的步长，padding方式为SAME：边缘外自动补0，遍历相乘;结果返回一个Tensor，这个输出，就是我们常说的feature map
    # 详见https://blog.csdn.net/mao_xiao_feng/article/details/53444333
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')    
def max_pool_2x2(x):    
    # 池化操作；x为池化层输入，ksize为池化窗口的大小 
    # 详见https://blog.csdn.net/mao_xiao_feng/article/details/53453926
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')    

# 二，定义输入输出结构  

# 声明一个占位符，None表示输入图片的数量不定，28*28图片分辨率  
xs = tf.placeholder(tf.float32, [None, 28*28])   
# 类别是0-9总共10个类别，对应输出分类结果  
ys = tf.placeholder(tf.float32, [None, 10])    
# x_image又把xs reshape成了28*28*1的形状，因为是灰色图片，所以通道是1.作为训练时的input，-1代表图片数量不定  
x_image = tf.reshape(xs, [-1, 28, 28, 1])   


# 三，搭建网络(核心步骤) 

## 第一层卷积操作/池化操作 ##  
# 定义32个5*5的通道为1的卷积核  
W_conv1 = weight_variable([5, 5, 1, 32])   
# 定义32个偏置值，每个卷积核对应一个偏置量
b_conv1 = bias_variable([32])    
# 执行卷积操作并加上偏执量，激活函数为ReLU函数，得到卷积结果（32-5+1）28x28x32  
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)    
# 执行池化操作，得到池化结果14x14x32  
h_pool1 = max_pool_2x2(h_conv1)   

## 第二层卷积/池化操作 ##     
# 64个5*5的通道为32的卷积核  
w_conv2 = weight_variable([5,5,32,64])   
# 64个偏置值  
b_conv2  = bias_variable([64])   
# 卷积结果14x14x64，注意此时输入为上一层的输出结果h_pool1 
h_conv2 = tf.nn.relu(conv2d(h_pool1,w_conv2)+b_conv2)    
# 池化结果7x7x64  
h_pool2 = max_pool_2x2(h_conv2)    

## 第三层全连接操作 ##  
W_fc1 = weight_variable([7*7*64, 1024])   
b_fc1 = bias_variable([1024])   
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64]) # 将第二层卷积池化结果reshape成二维数据，只有一行7*7*64个数据，即[n_samples, 7, 7, 64] ->> [n_samples, 7*7*64]     
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)   

## dropout操作，降低过拟合 ##
keep_prob = tf.placeholder(tf.float32)   
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

## 第四层输出操作 ##  
# 二维张量，1*1024矩阵卷积，共10个卷积，对应我们开始的ys长度为10  
W_fc2 = weight_variable([1024, 10])    
b_fc2 = bias_variable([10])    
# softmax输出，得到最终的分类结果1*1*10
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)   

# 四，定义损失函数和优化算法  
cross_entropy = -tf.reduce_sum(ys * tf.log(y_conv)) 
train_step = tf.train.GradientDescentOptimizer(0.0001).minimize(cross_entropy)

# 五，开始数据训练以及评测  
with tf.Session() as sess: 
    # tf.argmax(vector, 1)：返回的是vector中的最大值的索引号，如果vector是一个向量，那就返回一个值;
    # tf.equal(A, B)是对比这两个矩阵或者向量的相等的元素，如果是相等的那就返回True，反正返回False，返回的值的矩阵维度和A是一样的
    # tf.cast(x, dtype, name=None)将x转换成dtype类型
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(ys,1))  
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))  
    sess.run(tf.global_variables_initializer())  
    for i in range(2000):  
        batch = mnist.train.next_batch(50)  
        sess.run(train_step,feed_dict={xs: batch[0], ys: batch[1], keep_prob: 0.5})  
        if i%100 == 0:  
            train_accuracy = accuracy.eval(feed_dict={xs:batch[0], ys: batch[1], keep_prob: 1.0})  
            print("step %d, training accuracy %g"%(i, train_accuracy))  
    print("test accuracy %g"%accuracy.eval(feed_dict={xs: mnist.test.images, ys: mnist.test.labels, keep_prob: 1.0})) 

step 0, training accuracy 0.08
step 100, training accuracy 0.7
step 200, training accuracy 0.88
step 300, training accuracy 0.98
step 400, training accuracy 0.94
step 500, training accuracy 0.92
step 600, training accuracy 0.98
step 700, training accuracy 0.94
step 800, training accuracy 0.96
step 900, training accuracy 0.98
step 1000, training accuracy 0.96
step 1100, training accuracy 0.9
step 1200, training accuracy 0.94
step 1300, training accuracy 0.96
step 1400, training accuracy 0.94
step 1500, training accuracy 0.96
step 1600, training accuracy 0.92
step 1700, training accuracy 0.96
step 1800, training accuracy 0.96
step 1900, training accuracy 1
test accuracy 0.9615
